In [55]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial.annotations import *



A pure‑Python ROBDD 



In [ ]:
from dataclasses import dataclass
from typing import Dict, Tuple, Optional, Iterable, List

# ---------- Node dataclass ----------
@dataclass(frozen=True)
@attribute(field="id")
class Node:
    id: int                    # stable integer id (for debugging / refs)
    v: Optional[str]         # None for constants; otherwise variable name
    lo: Optional["Node"]       # 0-edge (None for constants)
    hi: Optional["Node"]       # 1-edge (None for constants)

    def is_const(self) -> bool:
        return self.v is None


# Pre-create constants
FALSE_NODE = atomColor(selector="{x: Node | @num:(x.id) = 0}", value='red')(Node(id=0, v=None, lo=None, hi=None))
TRUE_NODE  = atomColor(selector="{x: Node | @num:(x.id) = 1}", value='blue')(Node(id=1, v=None, lo=None, hi=None))


@flag(name="hideDisconnected")
@hideAtom(selector="dict + tuple + NoneType + B") # Hide internal machinery.
@orientation(selector="{x, y : Node | x->y in (lo + hi)}", directions=["below"]) # Alternate, imagine left/right split.
@orientation(selector="{x : Node, y : str | x->y in v}", directions=["directlyRight"])
@orientation(selector="{x, y : Node | @num:(x.id) = 1 and @num:(y.id) = 0}", directions=["directlyRight"])
#@group(field="v", groupOn=1, addToGroup=0) ## Alternate presentation
@hideField(field="v") # Could also show, part of alternate presentation!
class BDD:
    """
    Minimal ROBDD manager where Nodes reference Nodes directly.
    - Canonicality via unique table (v, lo.id, hi.id)
    - Append-only variable ordering
    - Minimal boolean ops: neg, and, or
    """
    def __init__(self, ordering: Optional[Iterable[str]] = None):
        self._nodes: Dict[int, Node] = {0: FALSE_NODE, 1: TRUE_NODE}
        self._unique: Dict[Tuple[str, int, int], Node] = {}
        self._var2level: Dict[str, int] = {}
        self._level2var: List[str] = []
        self._next_id: int = 2
        self._ite_cache: Dict[Tuple[int, int, int], Node] = {}
        if ordering:
            for v in ordering:
                self.add_var(v)

    # ---- Readable views ----
    @property
    def nodes(self) -> Dict[int, Node]:
        return self._nodes

    @property
    def variables(self) -> Tuple[str, ...]:
        return tuple(self._level2var)

    @property
    def roots(self) -> Tuple[Node, ...]:
        """Return nodes that are not referenced as children (constants excluded)."""
        children = {n.lo.id for n in self._nodes.values() if not n.is_const()} | \
                   {n.hi.id for n in self._nodes.values() if not n.is_const()}
        return tuple(self._nodes[i] for i in self._nodes.keys() if i not in children and i > 1)

    # ---- Variables ----
    def add_var(self, v: str) -> None:
        if v not in self._var2level:
            self._var2level[v] = len(self._level2var)
            self._level2var.append(v)

    def v(self, name: str) -> Node:
        if name not in self._var2level:
            self.add_var(name)
        return self._mk(name, FALSE_NODE, TRUE_NODE)

    def vars(self, *names: str):
        return tuple(self.v(n) for n in names)

    # ---- Unique constructor ----
    def _mk(self, v: str, lo: Node, hi: Node) -> Node:
        if lo is hi:
            return lo
        key = (v, lo.id, hi.id)
        n = self._unique.get(key)
        if n is not None:
            return n
        node = Node(id=self._next_id, v=v, lo=lo, hi=hi)
        self._next_id += 1
        self._unique[key] = node
        self._nodes[node.id] = node
        return node

    # ---- Helpers for ITE ----
    def _level(self, v: Optional[str]) -> int:
        return self._var2level[v] if v is not None else len(self._level2var) + 1

    def _top(self, u: Node) -> Optional[str]:
        return u.v

    def _split(self, u: Node, v: str) -> Tuple[Node, Node]:
        if u.is_const():
            return (u, u)
        return (u.lo, u.hi) if u.v == v else (u, u)

    # ---- ITE ----
    def ite(self, i: Node, t: Node, e: Node) -> Node:
        key = (i.id, t.id, e.id)
        if key in self._ite_cache:
            return self._ite_cache[key]

        if i is TRUE_NODE:   res = t
        elif i is FALSE_NODE: res = e
        elif t is e:         res = t
        elif t is TRUE_NODE and e is FALSE_NODE:  # ITE(i,1,0) == i
            res = i
        else:
            v = min((self._level(self._top(u)), self._top(u)) for u in (i, t, e))[1]
            assert v is not None
            i0, i1 = self._split(i, v)
            t0, t1 = self._split(t, v)
            e0, e1 = self._split(e, v)
            lo = self.ite(i0, t0, e0)
            hi = self.ite(i1, t1, e1)
            res = self._mk(v, lo, hi)

        self._ite_cache[key] = res
        return res

    # ---- Minimal boolean basis ----
    def neg(self, u: Node) -> Node:
        return self.ite(u, FALSE_NODE, TRUE_NODE)

    def op_and(self, a: Node, b: Node) -> Node:
        return self.ite(a, b, FALSE_NODE)

    def op_or(self, a: Node, b: Node) -> Node:
        return self.ite(a, TRUE_NODE, b)

    # ---- Evaluate ----
    def evaluate(self, u: Node, assignment: Dict[str, bool]) -> bool:
        while not u.is_const():
            u = u.hi if assignment.get(u.v, False) else u.lo
        return u is TRUE_NODE


# ---------- Tiny wrapper for notebook ergonomics ----------
class B:
    __slots__ = ("mgr", "node")
    def __init__(self, mgr: BDD, node: Node):
        self.mgr, self.node = mgr, node
    def __invert__(self):      return B(self.mgr, self.mgr.neg(self.node))
    def __and__(self, o: "B"): return B(self.mgr, self.mgr.op_and(self.node, o.node))
    def __or__(self, o: "B"):  return B(self.mgr, self.mgr.op_or(self.node, o.node))
    def evaluate(self, env):   return self.mgr.evaluate(self.node, env)
    def __repr__(self):        return f"B(Node id={self.node.id}, v={self.node.v})"



In [75]:


mgr = BDD(["x", "y"])
x_node, y_node = mgr.vars("x", "y")
x, y = B(mgr, x_node), B(mgr, y_node)

f = x & ~y
# print("Eval {x=1,y=0}:", f.evaluate({"x": True, "y": False}))  # True
# print("Variables:", mgr.variables)
# print("Root node ids:", [r.id for r in mgr.roots])
# for r in mgr.roots:
#     print("Root", r.id, "v:", r.v, "lo:", r.lo.id, "hi:", r.hi.id)

diagram(f)
